<a href="https://colab.research.google.com/github/issei/DaedalusForge/blob/main/automations/lan%C3%A7amento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração de Copy para Lançamentos com Agentes de IA em Grafo

### Objetivo Principal
Este notebook automatiza a criação de textos de marketing (copy) para o lançamento de um infoproduto. Utilizando um briefing detalhado como ponto de partida, uma rede de agentes de IA, orquestrada com **LangGraph**, gera de forma colaborativa e iterativa os principais ativos de comunicação necessários para a campanha.

### Arquitetura
A solução é construída sobre uma arquitetura que combina um Large Language Model (LLM) com um framework de orquestração de grafos e uma base de conhecimento vetorial (RAG).

- **LLM**: **Google Gemini 1.5 Flash**, um modelo rápido e eficiente para geração de texto.
- **Framework de Orquestração**: **LangGraph**, para criar um fluxo de trabalho cíclico e com estado, permitindo que os agentes colaborem e refinem o trabalho uns dos outros.
- **RAG (Retrieval-Augmented Generation)**: **DumplingAI** atua como uma base de conhecimento vetorial. O briefing do lançamento é indexado e pode ser consultado pelos agentes para garantir que a copy gerada seja consistente e alinhada à estratégia.

### Fluxo de Execução com LangGraph
O processo é gerenciado por um grafo de estados que coordena os agentes:

1.  **Configuração e Indexação**: O ambiente é preparado, e o briefing do lançamento é carregado e indexado na base de conhecimento (RAG).
2.  **Análise Inicial (Paralela)**: Três agentes especializados (`Dores & Promessas`, `Objeções & Quebras`, `Headlines & Ângulos`) analisam o briefing simultaneamente para extrair os insights fundamentais.
3.  **Consolidação de Contexto**: Um nó `Consolidador` reúne as análises iniciais, criando um "super contexto" enriquecido que servirá de base para a criação da copy.
4.  **Geração da Copy**: O agente de `Adaptação por Canais` utiliza o contexto enriquecido para criar as primeiras versões da copy para as diferentes plataformas (Email, Ads, etc.).
5.  **Ciclo de Revisão e Refinamento**:
    - Um agente `Crítico Revisor` avalia a copy gerada, comparando-a com o briefing original.
    - Se a copy for **"APROVADA"**, o fluxo termina.
    - Se for marcada para **"REFINAR"**, o agente de `Adaptação` recebe o feedback e gera uma nova versão, iniciando um novo ciclo de revisão.
6.  **Saída Final**: Após a aprovação, a versão final da copy é salva em arquivos `JSON` e `Markdown`, pronta para uso.

In [36]:
!pip -q install -U langchain langchain-google-genai google-generativeai httpx pydantic python-dotenv langgraph
print("✅ Dependências instaladas")

✅ Dependências instaladas


In [37]:
import os, json, httpx, re
from typing import List, Optional, Dict, Any
from datetime import datetime
from google.colab import userdata

# LangChain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

# Gemini via LangChain + SDK nativo
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai  # SDK oficial Google

# ======== PARAMS (edite aqui) ========
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
DUMPLING_API_KEY = userdata.get('DUMPLING_API_KEY')
DUMPLING_KB_ID = userdata.get('DUMPLING_KB_ID')
GEMINI_MODEL = "gemini-2.5-flash"  # @param {type:"string"}
TEMPERATURE = 0.6  # @param {type:"number"}
MAX_TOKENS = 2048

assert GOOGLE_API_KEY, "Defina GOOGLE_API_KEY"
assert DUMPLING_API_KEY, "Defina DUMPLING_API_KEY"
assert DUMPLING_KB_ID, "Defina DUMPLING_KB_ID"

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

# ======== Dumpling config ========
DUMPLING_BASE = "https://app.dumplingai.com/api/v1"
DUMPLING_HEADERS = {
    "Authorization": f"Bearer {DUMPLING_API_KEY}",
    "Content-Type": "application/json"
}

print("✅ Chaves configuradas")

✅ Chaves configuradas


In [38]:
# Definição do Estado do Grafo (AgentState)
from typing import TypedDict, Optional, Dict, Any

class AgentState(TypedDict):
    """Define a estrutura de dados que será compartilhada e modificada pelos nós do grafo."""
    briefing: Dict
    contexto_rag: str
    dores_promessas: Optional[Dict]
    objecoes_quebras: Optional[Dict]
    headlines_angulos: Optional[Dict]
    contexto_enriquecido: Optional[str]
    copy_por_canal: Optional[Dict]
    revisao_critico: Optional[str]
    tentativas_refinamento: int

In [39]:
# 🧩 Funções auxiliares – DumplingAI (httpx)
class DumplingError(RuntimeError):
    pass

def dumpling_add_to_kb(knowledge_base_id: str, name: str, content: str) -> Dict[str, Any]:
    """
    POST /knowledge-bases/add
    Body: { knowledgeBaseId, name, content }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/add-to-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/add"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "name": name,
        "content": content,
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Add KB falhou: {r.status_code} – {r.text}")
    return r.json()

def dumpling_search_kb(knowledge_base_id: str, query: str, result_count: int = 8) -> List[Dict[str, Any]]:
    """
    POST /knowledge-bases/query
    Body: { knowledgeBaseId, query, resultCount }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/search-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/query"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "query": query,
        "resultCount": max(1, min(int(result_count), 25))
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Query KB falhou: {r.status_code} – {r.text}")
    return r.json()

print("✅ Dumpling helpers prontos")

✅ Dumpling helpers prontos


In [40]:
# 🔎 Retriever customizado
class DumplingRetriever(BaseRetriever):
    knowledge_base_id: str
    top_k: int = 8

    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: Optional[CallbackManagerForRetrieverRun] = None,
    ) -> List[Document]:
        hits = dumpling_search_kb(self.knowledge_base_id, query, self.top_k)
        docs = []
        for h in hits:
            docs.append(Document(
                page_content=h.get("content") or "",
                metadata={
                    "id": h.get("id"),
                    "resource_name": h.get("resource_name"),
                    "similarity": h.get("similarity"),
                }
            ))
        return docs

retriever = DumplingRetriever(knowledge_base_id=DUMPLING_KB_ID, top_k=8)
print("✅ Retriever pronto")

✅ Retriever pronto


In [41]:
BRIEFING_JSON = r'''
{
  "briefing_lancamento": {
    "infoproduto": {
      "nome": "Mentoria de Desenvolvimento Inteligente",
      "produtor": "Mauricio Issei",
      "preco": 99997.00,
      "formato": "Mentoria Individual",
      "descricao": "Mentoria individual para desenvolver arquiteturas de soluções complexas"
    },
    "publico_alvo": {
      "demografia": "Empreendedores digitais iniciantes, profissionais de tecnologia",
      "problema_principal": "Dificuldade em resolver problemas, falta de método para desenvolvimento, baixo faturamento.",
      "transformacao_principal": "Criar soluções de alta qualidade e escalável do zero, alcançando faturamento de 6 ou 7 dígitos.",
      "objecoes_comuns": [
        "Não tenho conhecimento suficiente",
        "O preço é muito alto",
        "Não tenho tempo para aplicar o método"
      ]
    },
    "posicionamento": {
      "diferencial_competitivo": "Único método que combina estratégia de arquitetura de soluções com o poder do desenvolvimento com ferramentas de IA.",
      "tom_de_voz": "Autoridade, inspirador, prático",
      "gatilhos_mentais": [
        "Autoridade",
        "Prova Social",
        "Escassez",
        "Reciprocidade"
      ]
    },
    "estrategia_lancamento": {
      "tipo_lancamento": "Semente",
      "meta_campanha": "Vender 500 unidades e faturar R$ 500.000",
      "datas_chave": {
        "inicio_campanha": "2025-09-15",
        "abertura_carrinho": "2025-09-22",
        "fechamento_carrinho": "2025-09-29"
      },
      "canais": [
        "Email Marketing",
        "Meta Ads",
        "Instagram Stories",
        "YouTube"
      ]
    }
  }
}
'''

briefing = json.loads(BRIEFING_JSON)
print("✅ Briefing carregado")

✅ Briefing carregado


In [42]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Estilo para os campos de texto, para garantir que as descrições não sejam cortadas
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')

#--------------------------------------------------------------------
# 1. WIDGETS PARA "INFOPRODUTO"
#--------------------------------------------------------------------
info_data = briefing['briefing_lancamento']['infoproduto']
w_info_nome = widgets.Text(value=info_data['nome'], description='Nome:', style=style, layout=layout)
w_info_produtor = widgets.Text(value=info_data['produtor'], description='Produtor:', style=style, layout=layout)
w_info_preco = widgets.FloatText(value=info_data['preco'], description='Preço (R$):', style=style, layout=layout)
w_info_formato = widgets.Text(value=info_data['formato'], description='Formato:', style=style, layout=layout)
w_info_descricao = widgets.Textarea(value=info_data['descricao'], description='Descrição:', style=style, layout=layout)

#--------------------------------------------------------------------
# 2. WIDGETS PARA "PÚBLICO ALVO"
#--------------------------------------------------------------------
pa_data = briefing['briefing_lancamento']['publico_alvo']
w_pa_demografia = widgets.Textarea(value=pa_data['demografia'], description='Demografia:', style=style, layout=layout)
w_pa_problema = widgets.Textarea(value=pa_data['problema_principal'], description='Problema Principal:', style=style, layout=layout)
w_pa_transformacao = widgets.Textarea(value=pa_data['transformacao_principal'], description='Transformação Principal:', style=style, layout=layout)
w_pa_objecoes = widgets.Textarea(value='\n'.join(pa_data['objecoes_comuns']), description='Objeções Comuns (uma por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# 3. WIDGETS PARA "POSICIONAMENTO"
#--------------------------------------------------------------------
pos_data = briefing['briefing_lancamento']['posicionamento']
w_pos_diferencial = widgets.Textarea(value=pos_data['diferencial_competitivo'], description='Diferencial Competitivo:', style=style, layout=layout)
w_pos_tom = widgets.Text(value=pos_data['tom_de_voz'], description='Tom de Voz:', style=style, layout=layout)
w_pos_gatilhos = widgets.Textarea(value='\n'.join(pos_data['gatilhos_mentais']), description='Gatilhos Mentais (um por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# 4. WIDGETS PARA "ESTRATÉGIA DE LANÇAMENTO"
#--------------------------------------------------------------------
estr_data = briefing['briefing_lancamento']['estrategia_lancamento']
w_estr_tipo = widgets.Text(value=estr_data['tipo_lancamento'], description='Tipo de Lançamento:', style=style, layout=layout)
w_estr_meta = widgets.Text(value=estr_data['meta_campanha'], description='Meta da Campanha:', style=style, layout=layout)
w_estr_inicio = widgets.DatePicker(description='Início da Campanha:', value=datetime.strptime(estr_data['datas_chave']['inicio_campanha'], '%Y-%m-%d'), style=style)
w_estr_abertura = widgets.DatePicker(description='Abertura do Carrinho:', value=datetime.strptime(estr_data['datas_chave']['abertura_carrinho'], '%Y-%m-%d'), style=style)
w_estr_fechamento = widgets.DatePicker(description='Fechamento do Carrinho:', value=datetime.strptime(estr_data['datas_chave']['fechamento_carrinho'], '%Y-%m-%d'), style=style)
w_estr_canais = widgets.Textarea(value='\n'.join(estr_data['canais']), description='Canais (um por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# MONTAGEM DA INTERFACE COM ACORDEÃO
#--------------------------------------------------------------------
# Agrupa os widgets de cada seção em caixas verticais (VBox)
box_infoproduto = widgets.VBox([w_info_nome, w_info_produtor, w_info_preco, w_info_formato, w_info_descricao])
box_publico_alvo = widgets.VBox([w_pa_demografia, w_pa_problema, w_pa_transformacao, w_pa_objecoes])
box_posicionamento = widgets.VBox([w_pos_diferencial, w_pos_tom, w_pos_gatilhos])
box_estrategia = widgets.VBox([w_estr_tipo, w_estr_meta, w_estr_inicio, w_estr_abertura, w_estr_fechamento, w_estr_canais])

# Cria o acordeão e define as seções
accordion = widgets.Accordion(children=[box_infoproduto, box_publico_alvo, box_posicionamento, box_estrategia])
accordion.set_title(0, '🚀 Infoproduto')
accordion.set_title(1, '🎯 Público Alvo')
accordion.set_title(2, '📣 Posicionamento')
accordion.set_title(3, '🗓️ Estratégia de Lançamento')

# Botão para salvar as alterações
button = widgets.Button(description="✅ Atualizar Briefing", button_style='success', layout=widgets.Layout(width='200px', margin='10px 0 0 0'))
output = widgets.Output() # Área para exibir mensagens de status

# Função que será chamada quando o botão for clicado
def on_button_clicked(b):
    # Atualiza o dicionário 'briefing' com os valores dos widgets
    b_lanc = briefing['briefing_lancamento']
    b_lanc['infoproduto']['nome'] = w_info_nome.value
    b_lanc['infoproduto']['produtor'] = w_info_produtor.value
    b_lanc['infoproduto']['preco'] = w_info_preco.value
    b_lanc['infoproduto']['formato'] = w_info_formato.value
    b_lanc['infoproduto']['descricao'] = w_info_descricao.value

    b_lanc['publico_alvo']['demografia'] = w_pa_demografia.value
    b_lanc['publico_alvo']['problema_principal'] = w_pa_problema.value
    b_lanc['publico_alvo']['transformacao_principal'] = w_pa_transformacao.value
    b_lanc['publico_alvo']['objecoes_comuns'] = w_pa_objecoes.value.split('\n')

    b_lanc['posicionamento']['diferencial_competitivo'] = w_pos_diferencial.value
    b_lanc['posicionamento']['tom_de_voz'] = w_pos_tom.value
    b_lanc['posicionamento']['gatilhos_mentais'] = w_pos_gatilhos.value.split('\n')

    b_lanc['estrategia_lancamento']['tipo_lancamento'] = w_estr_tipo.value
    b_lanc['estrategia_lancamento']['meta_campanha'] = w_estr_meta.value
    b_lanc['estrategia_lancamento']['datas_chave']['inicio_campanha'] = w_estr_inicio.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['datas_chave']['abertura_carrinho'] = w_estr_abertura.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['datas_chave']['fechamento_carrinho'] = w_estr_fechamento.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['canais'] = w_estr_canais.value.split('\n')

    # Exibe uma mensagem de sucesso
    with output:
        clear_output()
        print(f"✔️ Briefing atualizado com sucesso às {datetime.now().strftime('%H:%M:%S')}!")
        # Opcional: Imprime o JSON atualizado para verificação
        # print(json.dumps(briefing, indent=2, ensure_ascii=False))


# Associa a função ao evento de clique do botão
button.on_click(on_button_clicked)

# Exibe a interface
display(accordion, button, output)

Accordion(children=(VBox(children=(Text(value='Mentoria de Desenvolvimento Inteligente', description='Nome:', …

Button(button_style='success', description='✅ Atualizar Briefing', layout=Layout(margin='10px 0 0 0', width='2…

Output()

In [43]:
# (Opcional) Upload de arquivo JSON via Colab
try:
    from google.colab import files  # type: ignore
    print("Selecione um arquivo .json (ou pule este passo).")
    up = files.upload()
    if up:
        fname = list(up.keys())[0]
        BRIEFING_JSON = open(fname, "r", encoding="utf-8").read()
        briefing = json.loads(BRIEFING_JSON)
        print("✅ JSON carregado de arquivo:", fname)
except Exception:
    print("Upload não disponível fora do Colab (ok).")

Selecione um arquivo .json (ou pule este passo).


In [44]:
def canonicalize_briefing_to_text(briefing: Dict[str, Any]) -> str:
    b    = briefing.get("briefing_lancamento", {})
    inf  = b.get("infoproduto", {})
    pub  = b.get("publico_alvo", {})
    pos  = b.get("posicionamento", {})
    est  = b.get("estrategia_lancamento", {})
    datas= est.get("datas_chave", {})

    linhas = []
    linhas.append("# Briefing de Lançamento — Canonicalizado")
    linhas.append("## Produto")
    linhas.append(f"Nome: {inf.get('nome','')} | Produtor: {inf.get('produtor','')} | Preço: {inf.get('preco','')} | Formato: {inf.get('formato','')}")
    linhas.append(f"Descrição: {inf.get('descricao','')}")
    linhas.append("\n## Público-alvo & Persona")
    linhas.append(f"Demografia/Psicografia: {pub.get('demografia','')}")
    linhas.append(f"Dor principal: {pub.get('problema_principal','')}")
    linhas.append(f"Transformação: {pub.get('transformacao_principal','')}")
    if pub.get("objecoes_comuns"):
        linhas.append("Objeções comuns:")
        for o in pub["objecoes_comuns"]:
            linhas.append(f"- {o}")
    linhas.append("\n## Posicionamento & Diferencial")
    linhas.append(f"USP: {pos.get('diferencial_competitivo','')}")
    linhas.append(f"Tom de voz: {pos.get('tom_de_voz','')}")
    if pos.get("gatilhos_mentais"):
        linhas.append("Gatilhos prioritários: " + ", ".join(pos["gatilhos_mentais"]))
    linhas.append("\n## Estratégia de Lançamento")
    linhas.append(f"Tipo: {est.get('tipo_lancamento','')} | Meta: {est.get('meta_campanha','')}")
    linhas.append(f"Período/Datas: início={datas.get('inicio_campanha','')} | abertura={datas.get('abertura_carrinho','')} | fechamento={datas.get('fechamento_carrinho','')}")
    if est.get("canais"):
        linhas.append("Canais: " + ", ".join(est["canais"]))
    return "\n".join(linhas)

canonical_text = canonicalize_briefing_to_text(briefing)
kb_name = f"Briefing — {datetime.utcnow().isoformat()}"

resp_add = dumpling_add_to_kb(DUMPLING_KB_ID, kb_name, canonical_text)
print("✅ Briefing indexado na KB com nome:", resp_add.get("name"))

/tmp/ipython-input-3227626404.py:35: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  kb_name = f"Briefing — {datetime.utcnow().isoformat()}"


✅ Briefing indexado na KB com nome: Briefing — 2025-09-21T00:20:41.936523


In [45]:
llm = ChatGoogleGenerativeAI(
    model=GEMINI_MODEL,
    temperature=TEMPERATURE
)

SYSTEM_BASE = (
    "Você é parte de uma REDE DE AGENTES especialista em copy para lançamentos de infoprodutos. "
    "Use o TOM DE VOZ do briefing. Foque em clareza, estratégia e ativos prontos para usar. "
    "Respeite a persona (dores, desejos) e o posicionamento (USP)."
)

def join_docs(docs: List[Document]) -> str:
    blocos = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("resource_name") or d.metadata.get("id") or f"doc{i}"
        blocos.append(f"[Fonte: {src}]\n{d.page_content}")
    return "\n\n".join(blocos)

def build_rag_context(brief: Dict[str, Any]) -> str:
    b   = brief.get("briefing_lancamento", {})
    pub = b.get("publico_alvo", {})
    pos = b.get("posicionamento", {})
    est = b.get("estrategia_lancamento", {})

    queries = []
    if pub.get("problema_principal"):
        queries.append(pub["problema_principal"])
    if pub.get("transformacao_principal"):
        queries.append(pub["transformacao_principal"])
    if pos.get("diferencial_competitivo"):
        queries.append(pos["diferencial_competitivo"])
    if est.get("tipo_lancamento"):
        queries.append(f"táticas de lançamento {est['tipo_lancamento']}")

    if not queries:
        queries = ["briefing do produto"]

    docs_all: List[Document] = []
    for q in queries:
        docs_all.extend(retriever.get_relevant_documents(q))

    # dedup simples
    seen, uniq = set(), []
    for d in docs_all:
        key = (d.page_content[:120], d.metadata.get("id"))
        if key not in seen:
            seen.add(key)
            uniq.append(d)

    return join_docs(uniq[:12])

contexto = build_rag_context(briefing)
briefing_str = json.dumps(briefing, ensure_ascii=False, indent=2)

# ===== Agente 1: Dores & Promessas =====
prompt_dores = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Gere DORES priorizadas e PROMESSAS (transformações)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: dores[], promessas[].")
])
chain_dores = prompt_dores | llm

# ===== Agente 2: Objeções & Quebras =====
prompt_obj = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Liste as OBJEÇÕES mais prováveis e QUEBRAS (respostas estratégicas)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: objecoes[], quebras[].")
])
chain_obj = prompt_obj | llm

# ===== Agente 3: Headlines & Ângulos =====
prompt_head = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Proponha HEADLINES e ÂNGULOS criativos prontos para testes."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: headlines[], angulos[].")
])
chain_head = prompt_head | llm

# ===== Agente 4: Adaptação por Canais =====
prompt_canais = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para Canais: Email, Stories IG, Meta Ads, YouTube/VSL."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: email.sequence[], stories.scripts[], ads.variacoes[], vsl.outline[].")
])
chain_canais = prompt_canais | llm

print("✅ Prompts e cadeias prontos")

✅ Prompts e cadeias prontos


In [46]:
# 3. Conversão dos Agentes em Nós
def node_dores_promessas(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de dores e promessas."""
    print("Executando nó de Dores & Promessas...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_dores.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"dores_promessas": safe_json(result)}

def node_objecoes_quebras(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de objeções e quebras."""
    print("Executando nó de Objeções & Quebras...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_obj.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"objecoes_quebras": safe_json(result)}

def node_headlines_angulos(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de headlines e ângulos."""
    print("Executando nó de Headlines & Ângulos...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_head.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"headlines_angulos": safe_json(result)}

In [47]:
# 4. Criação de Novos Nós Colaborativos

def node_consolidador(state: AgentState) -> Dict[str, Any]:
    """Concatena as saídas dos nós iniciais para criar um contexto enriquecido."""
    print("Executando nó Consolidador...")

    contexto_enriquecido = (
        "## Dores e Promessas\n"
        f"{json.dumps(state['dores_promessas'], ensure_ascii=False, indent=2)}\n\n"
        "## Objeções e Quebras\n"
        f"{json.dumps(state['objecoes_quebras'], ensure_ascii=False, indent=2)}\n\n"
        "## Headlines e Ângulos\n"
        f"{json.dumps(state['headlines_angulos'], ensure_ascii=False, indent=2)}"
    )

    return {"contexto_enriquecido": contexto_enriquecido}

# Modifica o prompt do agente de canais para usar o contexto enriquecido e a revisão
prompt_canais_refinado = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para Canais usando o CONTEXTO ENRIQUECIDO. Se houver uma REVISÃO, aplique as melhorias sugeridas."),
    ("human",
     "Briefing Original (JSON):\n{briefing}\n\n"
     "Contexto Enriquecido (gerado pelos agentes anteriores):\n{contexto_enriquecido}\n\n"
     "Revisão Anterior (se houver):\n{revisao_critico}\n\n"
     "Gere a copy adaptada para os canais. Retorne JSON com campos: email.sequence[], stories.scripts[], ads.variacoes[], vsl.outline[].")
])
chain_canais_refinado = prompt_canais_refinado | llm

def node_adaptacao_canais(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de adaptação por canais usando o contexto enriquecido."""
    print("Executando nó de Adaptação por Canais...")

    # Incrementa a tentativa
    tentativas = state.get('tentativas_refinamento', 0) + 1

    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    revisao = state.get('revisao_critico') or "Nenhuma" # Garante que não seja None

    result = chain_canais_refinado.invoke({
        "briefing": briefing_str,
        "contexto_enriquecido": state['contexto_enriquecido'],
        "revisao_critico": revisao
    })

    return {"copy_por_canal": safe_json(result), "tentativas_refinamento": tentativas}

# Novo agente: Crítico Revisor
prompt_critico = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um CRÍTICO DE MARKETING sênior e exigente. Sua tarefa é revisar a copy gerada para um lançamento. "
     "Seu feedback deve ser direto, acionável e baseado estritamente no briefing original. "
     "Se a copy estiver excelente e alinhada, responda apenas com a palavra 'APROVADO'. "
     "Se precisar de ajustes, comece com a palavra 'REFINAR:' e liste em bullets as mudanças necessárias. Seja específico. Ex: 'REFINAR: O tom de voz no email está muito informal, mude para algo mais de autoridade. Os stories precisam de um CTA mais claro.'"),
    ("human",
     "Briefing Original (JSON):\n{briefing}\n\n"
     "Copy Gerada para Revisão (JSON):\n{copy_por_canal}\n\n"
     "Avalie a copy. Responda 'APROVADO' ou 'REFINAR:' com suas sugestões.")
])
chain_critico = prompt_critico | llm

def node_critico_revisor(state: AgentState) -> Dict[str, Any]:
    """Executa o agente crítico para revisar a copy gerada."""
    print("Executando nó Crítico Revisor...")

    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    copy_str = json.dumps(state['copy_por_canal'], ensure_ascii=False, indent=2)

    result = chain_critico.invoke({
        "briefing": briefing_str,
        "copy_por_canal": copy_str
    })

    return {"revisao_critico": result.content}

print("✅ Novos nós colaborativos prontos")

✅ Novos nós colaborativos prontos


In [48]:
from langgraph.graph import StateGraph, END

# 5. Construção e Conexão do Grafo

def decidir_pos_critica(state: AgentState) -> str:
    """
    Decide o próximo passo após a revisão do crítico.
    Se a copy for aprovada ou o limite de tentativas for atingido, o fluxo termina.
    Caso contrário, volta para o nó de adaptação para refinamento.
    """
    print("Executando nó de Decisão Pós-Crítica...")
    revisao = state.get('revisao_critico', '')
    tentativas = state.get('tentativas_refinamento', 0)

    if "APROVADO" in revisao or tentativas >= 2:
        print("Decisão: Fim do ciclo.")
        return "fim"
    else:
        print(f"Decisão: Refinar (Tentativa {tentativas + 1}).")
        return "refinar"

# Montagem do grafo
workflow = StateGraph(AgentState)

# Adiciona os nós ao grafo
workflow.add_node("dores_promessas", node_dores_promessas)
workflow.add_node("objecoes_quebras", node_objecoes_quebras)
workflow.add_node("headlines_angulos", node_headlines_angulos)
workflow.add_node("consolidador", node_consolidador)
workflow.add_node("adaptacao_canais", node_adaptacao_canais)
workflow.add_node("critico_revisor", node_critico_revisor)

# Define o ponto de partida
# Os 3 primeiros nós são executados em paralelo
workflow.set_entry_point("dores_promessas")
workflow.set_entry_point("objecoes_quebras")
workflow.set_entry_point("headlines_angulos")


# Define as arestas (conexões)
workflow.add_edge("dores_promessas", "consolidador")
workflow.add_edge("objecoes_quebras", "consolidador")
workflow.add_edge("headlines_angulos", "consolidador")

workflow.add_edge("consolidador", "adaptacao_canais")
workflow.add_edge("adaptacao_canais", "critico_revisor")

# Adiciona a aresta condicional para o ciclo de feedback
workflow.add_conditional_edges(
    "critico_revisor",
    decidir_pos_critica,
    {
        "refinar": "adaptacao_canais",
        "fim": END
    }
)

print("✅ Grafo construído")

✅ Grafo construído


In [49]:
# 6. Compilação e Execução
app = workflow.compile()

# Define o estado inicial para a execução
initial_state = AgentState(
    briefing=briefing,
    contexto_rag=contexto,
    dores_promessas=None,
    objecoes_quebras=None,
    headlines_angulos=None,
    contexto_enriquecido=None,
    copy_por_canal=None,
    revisao_critico=None,
    tentativas_refinamento=0
)

# Invoca o grafo
print("\n🚀 Iniciando a execução do grafo de agentes...\n")
final_state = app.invoke(initial_state)

print("\n\n✅ Execução do grafo concluída!\n")
print("="*50)
print("\n📄 Resultado Final (Copy Aprovada):\n")
print(json.dumps(final_state['copy_por_canal'], ensure_ascii=False, indent=2))

# Salva o resultado final em arquivos
assets_final = final_state['copy_por_canal']
os.makedirs("outputs_langgraph", exist_ok=True)
with open("outputs_langgraph/assets_final.json", "w", encoding="utf-8") as f:
    json.dump(assets_final, f, ensure_ascii=False, indent=2)

with open("outputs_langgraph/assets_final.md", "w", encoding="utf-8") as f:
    f.write("# Assets de Lançamento (Gerados com LangGraph)\n\n")
    f.write(f"```json\n{json.dumps(assets_final, ensure_ascii=False, indent=2)}\n```\n\n")

print("\n\n✅ Resultados salvos em 'outputs_langgraph/assets_final.json' e 'outputs_langgraph/assets_final.md'")


🚀 Iniciando a execução do grafo de agentes...

Executando nó de Dores & Promessas...
Executando nó de Headlines & Ângulos...
Executando nó de Objeções & Quebras...
Executando nó Consolidador...
Executando nó de Adaptação por Canais...
Executando nó Crítico Revisor...
Executando nó de Decisão Pós-Crítica...
Decisão: Refinar (Tentativa 2).
Executando nó de Adaptação por Canais...
Executando nó Crítico Revisor...
Executando nó de Decisão Pós-Crítica...
Decisão: Fim do ciclo.


✅ Execução do grafo concluída!


📄 Resultado Final (Copy Aprovada):

{
  "raw": "Com base no briefing, contexto enriquecido e aplicando as revisões solicitadas, segue a copy adaptada para os canais de lançamento:\n\n```json\n{\n  \"email\": {\n    \"sequence\": [\n      {\n        \"nome\": \"Email 1: O Fim da Estagnação e o Início da Arquitetura Inteligente\",\n        \"assunto\": \"Chega de Soluções 'Remendadas': Sua Virada para 6 e 7 Dígitos Começa Aqui.\",\n        \"pre_header\": \"Mauricio Issei revela o mét

In [51]:
def safe_json(resp) -> Any:
    text = resp.content if hasattr(resp, "content") else str(resp)
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{[\s\S]*\}\s*$", text)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                pass
        return {"raw": text}

assets = {
    "dores_promessas":  safe_json(outputs["dores"]),
    "objeções_quebras": safe_json(outputs["obj"]),
    "headlines_angulos":safe_json(outputs["head"]),
    "por_canais":       safe_json(outputs["canais"]),
}

import os
os.makedirs("outputs", exist_ok=True)
with open("outputs/assets.json", "w", encoding="utf-8") as f:
    json.dump(assets, f, ensure_ascii=False, indent=2)

with open("outputs/assets.md", "w", encoding="utf-8") as f:
    f.write("# Assets de Lançamento (gerados)\n\n")
    for k,v in assets.items():
        f.write(f"## {k}\n```json\n{json.dumps(v, ensure_ascii=False, indent=2)}\n```\n\n")

print("✅ Salvo em outputs/assets.json e outputs/assets.md")

✅ Salvo em outputs/assets.json e outputs/assets.md


In [53]:
# @title ⬆️ (Opcional) Enviar os assets gerados para a Knowledge Base
PUSH_ASSETS_TO_KB = True  # @param {type:"boolean"}

if PUSH_ASSETS_TO_KB:
    content_assets = open("outputs/assets.md", "r", encoding="utf-8").read()
    resp_add_assets = dumpling_add_to_kb(
        DUMPLING_KB_ID,
        name=f"Assets Gerados — {datetime.utcnow().isoformat()}",
        content=content_assets
    )
    print("✅ Assets enviados para a KB como:", resp_add_assets.get("name"))
else:
    print("Pulando envio à KB (PUSH_ASSETS_TO_KB=False).")

/tmp/ipython-input-3427886406.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  name=f"Assets Gerados — {datetime.utcnow().isoformat()}",


✅ Assets enviados para a KB como: Assets Gerados — 2025-09-21T00:24:33.617760
